In [1]:
import copy
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import tensorflow as tf
from google.cloud import bigquery

2024-09-16 13:40:42.924995: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
NIR_PATH = "gs://etldata-prod-search-ranking-data-hkwv8r/data/shared/neural_ir/30d/large_voc_huge_hidden_assym/models/2024_09_14/training_dir/checkpoints/saved_model"
nir_model = tf.saved_model.load(NIR_PATH)

In [3]:
def sample_pairs_from_date(date_str):
    query_str = f"""with lfb_clean as (
      SELECT 
        key AS listingId,
        IFNULL(verticaSellerBasics_shopName, "") shopName,
      FROM `etsy-ml-systems-prod.feature_bank_v2.listing_feature_bank_2024-09-16`
      where verticaSellerBasics_shopName is not null
      and verticaSellerBasics_shopName != ""
    ),
    results_pair as(
        SELECT distinct _date, query, listingId, requestUUID, target, 'web' rpSource 
        FROM `etsy-search-kubeflow-dev.behavioral_data.results_pair_fl_web`
        where _date = date('{date_str}')
        UNION ALL
        SELECT distinct _date, query, listingId, requestUUID, target, 'boe' rpSource 
        FROM `etsy-search-kubeflow-dev.behavioral_data.results_pair_fl_boe`
        where _date = date('{date_str}')
    )
    select 
      _date, query, requestUUID, target, rpSource,
      lfb_clean.*
    from results_pair
    join lfb_clean
    using (listingId)
    where query is not null
    and rand() > 0.9
    limit 50000
    """

    client = bigquery.Client(project="etsy-bigquery-adhoc-prod")
    query_job = client.query(query_str)
    rows = query_job.result()
    df = rows.to_dataframe()

    return df


def compute_nir_cosine_score(df, nir_model, batch_size=256):
    n_batches = df.shape[0] // batch_size + 1

    all_cos_scores = None
    
    for i in range(n_batches):
        start_idx = i * batch_size
        end_idx = min( (i+1) * batch_size, df.shape[0] )
        curr_df = df.iloc[start_idx:end_idx, :]

        query_embeddings = nir_model.signatures["embed_raw_queries"](
            tf.constant(list(curr_df['query']), tf.string)
        )["embedding"]

        shop_name_embeddings = nir_model.signatures["embed_raw_queries"](
            tf.constant(list(curr_df['shopName']), tf.string)
        )["embedding"]

        normalize_query = tf.math.l2_normalize(query_embeddings, axis=1)
        normalize_shop = tf.math.l2_normalize(shop_name_embeddings, axis=1)
        cos_score = tf.math.reduce_sum(tf.multiply(normalize_query, normalize_shop), axis=1)
        cos_score = tf.reshape(cos_score, [-1]).numpy()

        if all_cos_scores is None:
            all_cos_scores = cos_score
        else:
            all_cos_scores = np.concatenate((all_cos_scores, cos_score), axis=None)

    new_df = copy.deepcopy(df)
    new_df['query_shop_cosine'] = all_cos_scores
    return new_df


def get_shop_name_queries_examples(date_list, nir_model, threshold=0.8):
    output_df = None
    
    for date_str in tqdm(date_list):
        df = sample_pairs_from_date(date_str)
        new_df = compute_nir_cosine_score(df, nir_model, batch_size=512)
        new_df = new_df[new_df.query_shop_cosine >= threshold]

        if output_df is None:
            output_df = new_df
        else:
            output_df = pd.concat([output_df, new_df], ignore_index=True)

    return output_df

In [4]:
DATE_LIST = [d.strftime('%Y-%m-%d') for d in pd.date_range('2024-09-01','2024-09-10')]

In [5]:
df = get_shop_name_queries_examples(DATE_LIST, nir_model)

  0%|          | 0/10 [00:00<?, ?it/s]

/Users/yzhang/development/yzhang-adhoc-analysis/venv/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/yzhang/development/yzhang-adhoc-analysis/venv/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/yzhang/development/yzhang-adhoc-analysis/venv/lib/python3.1

In [6]:
df.shape

(2007, 8)

In [8]:
df[["query", "shopName"]].head(n=20)

,query,shopName
0,cutnsewsimply,cutnsewsimply
1,gruhomevn,GruhomeVN
2,LordsandLabradors,LordsandLabradors
3,klinehoneybeefarm,KlineHoneyBeeFarm
4,beerenapfel,Beerenapfel
5,jtrimming,Jtrimming
6,xoxokristen,XOXOKristen
7,oxosilver,OxoSilver
8,lausbub,Lausbub
9,jennfeli,JennFeli


In [9]:
len(df.shopName.unique())

1670

In [10]:
unique_listing_ids = list(df.listingId.unique())
len(unique_listing_ids)

1832

In [11]:
query_str = f"""with lfb as (
    select
        key AS listingId,
        IFNULL(verticaListings_title, "") listingTitle,
        IFNULL(verticaListings_description, "") listingDescription,
        IFNULL(verticaListings_taxonomyPath, "") listingTaxo,
        (SELECT STRING_AGG(element, ';') FROM UNNEST(kbAttributesV2_sellerAttributesV2.list)) AS listingAttributes,
        (SELECT STRING_AGG(element, ';') FROM UNNEST(kbMaterials_materialListingTags.list)) AS listingMaterialTags,
        IFNULL(verticaListings_tags, "") listingTags,
        IFNULL(verticaSellerBasics_shopName, "") shopName,
    FROM `etsy-ml-systems-prod.feature_bank_v2.listing_feature_bank_2024-09-16`
),
lfb_clean as (
    select *
    from lfb
    where listingTitle != ""
    and listingDescription != ""
    and listingAttributes != ""
    and shopName != ""
)
select *
from lfb_clean
where listingId in ({",".join([str(x) for x in unique_listing_ids])})
"""

client = bigquery.Client(project="etsy-bigquery-adhoc-prod")
query_job = client.query(query_str)
rows = query_job.result()
fb_df = rows.to_dataframe()

/Users/yzhang/development/yzhang-adhoc-analysis/venv/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [12]:
merged_df = pd.merge(df, fb_df, on="listingId", how="inner")

In [13]:
merged_df.shape

(1280, 15)

In [14]:
merged_df.to_csv("shop_name_query_real.csv")